# Assignment 4: kNN, Naive Bayes and Text Feature Selection

#### Full Name: April Hudspeth
#### Student ID: 995032557

For this assignment, we'll explore kNN and Naive Bayes further and look at different feature selection strategies. First of all, download the dataset for this assignment from Canvas and unpack into the A4 directory. 

The dataset is from the RCV1 v2 corpus which is a collection of news articles and category labels. There are 103 categories, and each document can lie in one or more categories. 

The sparse train and test matrices are encoded as nnz x 3 integer matrices, where nnz is the total number of words in all docs. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords


In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline

Now read the data files. 

In [2]:
iwords = np.loadtxt("data/words.imat.txt")          # training data matrix in nnz x 3 form - rows are (doc, word, count) triples
cats = np.loadtxt("data/cats.imat.txt")             # training labels in an ndocs x ncats matrix
tiwords = np.loadtxt("data/testwords.imat.txt")     # test data matrix in nnz x 3 form
tcats = np.loadtxt("data/testcats.imat.txt")        # testing labels in an ndocs x ncats matrix



iwords and tiwords encode the sparse train and test matrices. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords. The exact matrix type is CSR which stands for compressed sparse rows. Look up its description here: https://en.wikipedia.org/wiki/Sparse_matrix

In [3]:
fwords = iwords[:,2]
print fwords.shape
print iwords[:,1].shape
print iwords[:,0].shape
train = sp.csr_matrix((fwords, (iwords[:,1], iwords[:,0])))

ndocs = train.shape[0]
nwords = train.shape[1]

test = sp.csr_matrix((tiwords[:,2], (tiwords[:,1], tiwords[:,0])),shape=(4000,nwords))  # need to match the number of cols (words)
ntdocs = test.shape[0]



(801667,)
(801667,)
(801667,)


Although the dataset has 103 category labels, we'll focus on training a single category. We will use category 6, which is fairly evenly balanced between positives and negatives. 

In [4]:
cat6 = cats[:,6]
tcat6 = tcats[:,6]


This time we'll train Scikit-Learn's builtin kNN classifier. 

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knnc = KNeighborsClassifier(n_neighbors=3)

And then "fit" it to the data. Notice how fast this is! Of course, training kNN is basically a no-op, and predicting is where all the work lies:

In [6]:
knnc.fit(train, cat6)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           n_neighbors=3, p=2, weights='uniform')

Now lets try predicting. This will actually take some time, but hopefully only a few tens of seconds...

In [7]:
preds = knnc.predict(test);preds

array([ 0.,  1.,  1., ...,  0.,  0.,  0.])

Now we can compute the accuracy on the actual test set labels. 

In [8]:
np.mean(tcat6 == preds)

0.82225000000000004

Not bad, but kNN is very sensitive to the distance function as we argued in class. Let's try first normalizing each row of "train" and "test" by the L2-norm of that row, i.e. we will divide by the sqrt of the sum of the squared word counts in the row. Comparing the normalized documents should do much better. 

> Q1: Implement row normalization below. Compute the row norms first. Your implementation strategy is important for this to run in reasonable time. **Don't** try to construct an ndocs x ndocs dense matrix. You probably also don't want to iterate over all the docs or all the words. The fastest solution is to pre-multiply by a sparse diagonal matrix which contains the scale factors. No loops are necessary. Normalization should take a fraction of a second. 

In [9]:
from sklearn import preprocessing
from numpy.linalg import inv
from scipy.spatial.distance import pdist, squareform
# row_sums = np.array(train.sum(axis=1))[:,0]
# row_indices, col_indices = train.nonzero()


trainnorm = train.multiply(train).sum(1)
testnorm = test.multiply(test).sum(1)
trainfact = 1.0 / np.sqrt(np.max(trainnorm,1))
testfact = 1.0 / np.sqrt(np.max(testnorm,1));trainfact.shape
trainnmat = sp.spdiags(np.transpose(trainfact), 0, ndocs, ndocs) * train
testnmat = sp.spdiags(np.transpose(testfact), 0, ntdocs, ntdocs) * test
testnmat.shape # should be (10000,44718)


(4000, 31331)

Now train and evaluate a knn model using the normalized datasets:

In [10]:
knnc.fit(trainnmat, cat6)
preds = knnc.predict(testnmat);preds
np.mean(tcat6 == preds)     

0.90275000000000005

> Q2: What is the accuracy of your kNN classifier on the normalized data? **0.903**

## Naive Bayes Classification

Let's create a Multinomial Naive Bayes classifier. The multinomial distribution models each word in a document as being drawn from a category-specific probability distribution over the vocabulary. 

In [11]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

Now we can fit the NB model, which simply means tabulating the word counts for each label.

In [12]:
mnb.fit(train, cat6)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

...and predict the category labels for cat6:

In [13]:
preds = mnb.predict(test);preds

array([ 0.,  1.,  1., ...,  1.,  0.,  1.])

> Q3: Let's check the accuracy below. How did NB do compared to kNN? 


> ** The accuracy was the same for the kNN than the NB. **

In [14]:
np.mean(tcat6 == preds)

0.89749999999999996

## Frequency-Based Feature Selection

We argued that frequency-based feature selection is useful approach to culling features to reduce the size of a model. In some cases, accuracy improves as well. 

Let's construct a matrix rcounts which contains the counts of features from the training set. 

In [15]:
rcounts = (train>0).sum(0)
rcounts

matrix([[3728, 4438, 3488, ...,    1,    1,    1]])

Next we'll find the indices of non-zero elements of the predicate (rcounts >= count). These are the features that occurred at least count times. ii is the indices of these features.

We then extract the columns of the train and test matrices corresponding to those high-count features. The last line tells us how many features we kept.

In [16]:
import pandas as pd


threshold = 1
ii0 = np.asarray(np.nonzero(rcounts >= threshold)[1])
ii = ii0.reshape(ii0.size,)
strain = train[:,ii]
stest = test[:,ii]
ii.size


31331

Finally we run training and evaluation on train/test data restricted to those columns to get an accuracy score.

In [17]:
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

thresholds = [2,5,10,20]
li = [[1, ii.size, np.mean(tcat6 == preds)]]
for t in thresholds:
    threshold = t
    ii0 = np.asarray(np.nonzero(rcounts >= threshold)[1])
    ii = ii0.reshape(ii0.size,)
    strain = train[:,ii]
    stest = test[:,ii]
    ii.size
    mnb.fit(strain, cat6)
    preds = mnb.predict(stest);preds
    np.mean(tcat6 == preds)
    li.append([t, ii.size, np.mean(tcat6 == preds)])


df3 = pd.DataFrame(li,columns=['Count','Features', 'Score'])
df3

,Count,Features,Score
0,1,31331,0.89750
1,2,18330,0.89550
2,5,9632,0.89000
3,10,6169,0.88675
4,20,4046,0.88400


> Q4: For threshold counts of 1, 2, 5, 10, 20, tabulate the count, number of features kept, and the accuracy score. 

> Q5: How did frequency-based feature selection affect accuracy? How much was model size (proportional to words kept) reduced? 

>**The more features yielded higher accuracy than lower number of features. Model size was reduced by almost half each time. **

## Feature Selection Based on Statistical Tests

Let's try a more sophisticated method to select the (word) features. Since we have count data, the Chi-Squared test is appropriate. Remember that Chi-squared can be used on 2x2 contingency tables to determine if there is an interaction. In this case, we test if the label interacts with the feature. 

In [18]:
fcounts = np.zeros((4,nwords))           # holds the counts of various combinations of label and feature
expected = np.zeros((4,nwords))          # holds the predicted counts of those same combinations

In [39]:
mcat6 = cat6.reshape(1,ndocs)
fcounts[0,:] = mcat6 * (train > 0)                                   # label and word both = 1
fcounts[1,:] = (1-mcat6) * (train > 0)                               # label = 0, word = 1
fcounts[2,:] = np.sum(mcat6) * np.ones((1,nwords)) - fcounts[0,:]    # label = 1, word = 0
fcounts[3,:] = np.sum(1-mcat6) * np.ones((1,nwords)) - fcounts[1,:]  # label = 0, word = 0


Review the definition of the CHI-squared test here:
https://en.wikipedia.org/wiki/Chi-squared_test
Under the null hypothesis that category label doesn't matter, we expect to be able to compute 

In [40]:
fcounts[0,:]

array([ 1924.,  2273.,  2204., ...,     0.,     0.,     0.])

In [41]:
allf = np.sum(fcounts,axis=0)
pword0 = (fcounts[2,:] + fcounts[3,:]) / allf    # prob word = 0
pword1 = (fcounts[0,:] + fcounts[1,:]) / allf    # prob word = 1
pcat0 = (fcounts[1,:] + fcounts[3,:]) / allf     # prob label = 0
pcat1 = (fcounts[0,:] + fcounts[2,:]) / allf     # prob label = 1

If label and word occurence are independent, then the expected count should be the product of the total count and the corresponding probabilities:

> Q6: Implement the expected counts here:

In [48]:
expected[0,:] =  pcat1 * pword1 # expected count with label = 1, word = 1
expected[1,:] =  pcat0 * pword1  # expected count with label = 0, word = 1
expected[2,:] =  pcat1 * pword0 # expected count with label = 1, word = 0
expected[3,:] =  pcat0 * pword0 # expected count with label = 0, word = 0

To compute the chi-squared statistic value, we want the Sum of the squared differences between actual and predicted counts, normalized by the expected counts. The following two cells perform that calculation. We have to guard against divide-by-zero so we use a small lower-bound on the denominator.

In [49]:
diff = fcounts - expected
safeexpected = np.maximum(expected,1e-7)
diff

array([[  1.92382963e+03,   2.27279718e+03,   2.20384060e+03, ...,
         -4.57000000e-05,  -4.57000000e-05,  -4.57000000e-05],
       [  1.80379757e+03,   2.16475902e+03,   1.28381060e+03, ...,
          9.99945700e-01,   9.99945700e-01,   9.99945700e-01],
       [  2.64571337e+03,   2.29674582e+03,   2.36570240e+03, ...,
          4.56954305e+03,   4.56954305e+03,   4.56954305e+03],
       [  3.62565943e+03,   3.26469798e+03,   4.14564640e+03, ...,
          5.42845705e+03,   5.42845705e+03,   5.42845705e+03]])

> Q7: Implement the CHI-squared statistic value here

In [50]:
chisquaredstat =  np.square(diff) / safeexpected # implement the chisquared statistic value here
chisquaredstat.shape


(4, 31331)

We can then use the chi-squared statistic to choose the best features. The CHI-squared stastistic is a measure of interaction between a word feature and the label. The higher its value, the better that word should be as a predictor of the category. We set a threshold and take the words whose chi-squared statistic is above this value:

In [51]:
chisqthreshold = 1.5
ii0 = np.asarray(np.nonzero(chisquaredstat > chisqthreshold)[0])
ii = ii0.reshape(ii0.size,);ii.shape

(104691,)

Finally we use the indices of good features selected by the chi-squared filter to train and evaluate the Naive Bayes model.

In [ ]:
strain = train[:,ii]
stest = test[:,ii]
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

>(optional): Compare the accuracy of CHI-squared feature selection to the frequency-based selection you did earlier. For each frequency threshold (1,2,5,10,20), adjust the chisquared threshold to retain the smallest number of features that is at least as large as the number of features kept by that frequency threshold. Then compute the accuracy in the last cell for that threshold. Finally tabulate both frequency-based and chi-squared accuracies. That is, make a table whose columns are:
* freq threshold
* number of features retained by this frequency threshold
* chisquared threshold 
* number of features retained by this chisquared threshold (should be >= column 2)
* accuracy of frequency thresholded model
* accuracy of chisquared thresholded model
and whose rows are indexed by the frequency thresholds 1, 2, 5, 10, 20


>(optional): Which method has better accuracy for a given number of features? 

## Submit

Submit your A4 notebook on Canvas.